In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
from konlpy.tag import Mecab
import re
from konlpy.tag import Twitter
from sklearn.model_selection import train_test_split
twitter = Twitter()
from collections import namedtuple
from keras.preprocessing import sequence

/anaconda3/lib/python3.7/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
/anaconda3/lib/python3.7/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)
Using TensorFlow backend.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data

from torchtext import data
from torchtext import datasets

from ignite.engine import Engine, Events
from ignite.metrics import Accuracy, Loss, RunningAverage
from ignite.handlers import ModelCheckpoint, EarlyStopping
from ignite.contrib.handlers import ProgressBar
#import Early Stopping
from pytorchtools import EarlyStopping

In [3]:
dataset = pd.read_csv("/Users/SAEROM/slang_dict_twitter(1).csv", encoding='cp949')
#첫 5행을 불러옴
dataset.head() 

,word,frequency,dict,class
0,새끼,64,1,1
1,남자,37,1,0
2,진짜,36,1,0
3,여자,32,1,0
4,사람,27,1,0


In [4]:
train, test = train_test_split(dataset, test_size=0.2, shuffle = True)
train, valid = train_test_split(train, test_size=0.2, shuffle = True)
print(len(train), len(valid), len(test))

640 160 200


In [5]:
class Data(torch.utils.data.Dataset):
    def __init__(self, data):
        #self.data=[]
        #self.label=[]
        
        #word,freq,dict,tag라는 라벨을 갖고 있는 namedtuple 클래스를 생성.
        TaggedWord = namedtuple('TaggedWord', 'word freq dict tag')
        tagged_words = [TaggedWord(w, f, d, c) for w,f,d,c 
                              in data[["word", "frequency", "dict", "class"]].values]
        #print(tagged_words)
        data_x = []
        data_y = []
        
        
        for t in tagged_words:
            vector_=[]
            vector_.append(t.freq)
            vector_.append(t.dict)
            
        
            data_x.append(vector_)
            data_y.append(t.tag)
        #print(data_x)
        #print(data_y)

        #print(type(np.array(data_x)))
        #np.array(data_y)
        
        self.data = np.array(data_x)
        self.label = np.array(data_y)
        
        print("<데이터의 shape>")
        print(type(self.data))
        print(np.array(self.data).shape)
        print("<데이터 내용>") 
        print(torch.tensor(self.data[0], dtype=torch.float))
        print("<레이블 표시>") 
        print(self.label[0])
        print("<총 레이블 길이>") 
        print(len(self.label)) 
        
    def __getitem__(self, index):
        return torch.tensor(self.data[index], dtype=torch.float), self.label[index]
    
    def __len__(self):
        return len(self.data)

In [6]:
class DNN (nn.Module):
    def __init__(self):
        super (DNN, self).__init__ ()
        
        self.fc1 = nn.Sequential(
            nn.Linear (2,32),
            nn.ELU())
        self.fc2 = nn.Sequential(
            nn.Linear (32, 16),
            nn.ELU())
        self.fc3 = nn.Sequential(
            nn.Linear (16, 2),
            nn.ELU())
        self.dropout = nn.Dropout(0.5)
        
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        x = x.view (-1, 2)
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        
        x = self.softmax(x)
        
        return x
model = DNN()
print(model)

DNN(
  (fc1): Sequential(
    (0): Linear(in_features=2, out_features=32, bias=True)
    (1): ELU(alpha=1.0)
  )
  (fc2): Sequential(
    (0): Linear(in_features=32, out_features=16, bias=True)
    (1): ELU(alpha=1.0)
  )
  (fc3): Sequential(
    (0): Linear(in_features=16, out_features=2, bias=True)
    (1): ELU(alpha=1.0)
  )
  (dropout): Dropout(p=0.5)
  (softmax): Softmax()
)


In [7]:
# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = torch.optim.Adam(model.parameters())

In [8]:
from pytorchtools import EarlyStopping

In [9]:
def train_model (model, batch_size, patience, epochs): 
    
    ##################################################
    #train and validate model#
    ##################################################
    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 
  

    ###################
    # train the model #
    ###################  
    # initialize the early_stopping object
    early_stopping = EarlyStopping(patience=patience, verbose=True)
        
    for epoch in range(1, epochs+1):
        for batch, (data, label) in enumerate (loader_train, 1):
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            loss = criterion(output, label)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # record training loss
            train_losses.append(loss.item())
            
    ###################
    # validate the model #
    ###################  
        model.eval()
        for data, label in loader_valid:
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the loss
            loss = criterion(output, label)
            # record validation loss
            valid_losses.append(loss.item())
    
        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        
        epoch_len = len(str(epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'valid_loss: {valid_loss:.5f}')
        
        print(print_msg)
        
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        
        # early_stopping needs the validation loss to check if it has decresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
            
    return  model, avg_train_losses, avg_valid_losses

In [10]:
data_train = Data (data = train)
loader_train = torch.utils.data.DataLoader(data_train, batch_size = 20, shuffle = False)
data_test = Data (data = test)
loader_test = torch.utils.data.DataLoader(data_test, batch_size = 20, shuffle = False)
data_valid = Data (data = valid)
loader_valid = torch.utils.data.DataLoader(data_valid, batch_size = 20, shuffle = False)

<데이터의 shape>
<class 'numpy.ndarray'>
(640, 2)
<데이터 내용>
tensor([5., 1.])
<레이블 표시>
0
<총 레이블 길이>
640
<데이터의 shape>
<class 'numpy.ndarray'>
(200, 2)
<데이터 내용>
tensor([1., 1.])
<레이블 표시>
0
<총 레이블 길이>
200
<데이터의 shape>
<class 'numpy.ndarray'>
(160, 2)
<데이터 내용>
tensor([17.,  0.])
<레이블 표시>
1
<총 레이블 길이>
160


In [11]:
# early stopping patience; how long to wait after last time validation loss improved.
#patience = 20
model, train_loss, valid_loss = train_model(model, 
                                            batch_size=20,epochs=100 , patience = 20)

[  1/100] train_loss: 0.57664 valid_loss: 0.53274
Validation loss decreased (inf --> 0.532735).  Saving model ...
[  2/100] train_loss: 0.52579 valid_loss: 0.51844
Validation loss decreased (0.532735 --> 0.518436).  Saving model ...
[  3/100] train_loss: 0.51576 valid_loss: 0.51022
Validation loss decreased (0.518436 --> 0.510217).  Saving model ...


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[  4/100] train_loss: 0.50474 valid_loss: 0.49244
Validation loss decreased (0.510217 --> 0.492444).  Saving model ...
[  5/100] train_loss: 0.47556 valid_loss: 0.45592
Validation loss decreased (0.492444 --> 0.455920).  Saving model ...
[  6/100] train_loss: 0.44372 valid_loss: 0.43418
Validation loss decreased (0.455920 --> 0.434185).  Saving model ...
[  7/100] train_loss: 0.42825 valid_loss: 0.42528
Validation loss decreased (0.434185 --> 0.425278).  Saving model ...
[  8/100] train_loss: 0.42169 valid_loss: 0.42113
Validation loss decreased (0.425278 --> 0.421130).  Saving model ...
[  9/100] train_loss: 0.41834 valid_loss: 0.41892
Validation loss decreased (0.421130 --> 0.418924).  Saving model ...
[ 10/100] train_loss: 0.41643 valid_loss: 0.41757
Validation loss decreased (0.418924 --> 0.417573).  Saving model ...
[ 11/100] train_loss: 0.41527 valid_loss: 0.41668
Validation loss decreased (0.417573 --> 0.416682).  Saving model ...
[ 12/100] train_loss: 0.41450 valid_loss: 0.4160

[ 73/100] train_loss: 0.41174 valid_loss: 0.41332
Validation loss decreased (0.413318 --> 0.413316).  Saving model ...
[ 74/100] train_loss: 0.41174 valid_loss: 0.41331
Validation loss decreased (0.413316 --> 0.413314).  Saving model ...
[ 75/100] train_loss: 0.41174 valid_loss: 0.41331
Validation loss decreased (0.413314 --> 0.413312).  Saving model ...
[ 76/100] train_loss: 0.41174 valid_loss: 0.41331
Validation loss decreased (0.413312 --> 0.413311).  Saving model ...
[ 77/100] train_loss: 0.41174 valid_loss: 0.41331
Validation loss decreased (0.413311 --> 0.413309).  Saving model ...
[ 78/100] train_loss: 0.41174 valid_loss: 0.41331
Validation loss decreased (0.413309 --> 0.413308).  Saving model ...
[ 79/100] train_loss: 0.41173 valid_loss: 0.41331
Validation loss decreased (0.413308 --> 0.413307).  Saving model ...
[ 80/100] train_loss: 0.41173 valid_loss: 0.41331
Validation loss decreased (0.413307 --> 0.413305).  Saving model ...
[ 81/100] train_loss: 0.41173 valid_loss: 0.4133

In [12]:
def test_model(model, batch_size):
    # initialize lists to monitor test loss and accuracy
    test_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    model.eval() # prep model for evaluation

    for data, target in loader_test:
        if len(target.data) != batch_size:
            break
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update test loss 
        test_loss += loss.item()*data.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)
        # compare predictions to true label
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        # calculate test accuracy for each object class
        print(target)
        print(data)
        print(pred)
        for i in range(batch_size):
            label = target.data[i]
            #print('#############')
            #print(, end=' ')
            #print(target.data.view_as)
            #print(loader_test.dataset)
            #print("\t", label, "\t" , target.data.view_as(pred))
            class_correct[label] += correct[i].item()
            class_total[label] += 1

        # calculate and print avg test loss
        test_loss = test_loss/len(test)
        print('Test Loss: {:.6f}\n'.format(test_loss))

    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
                str(i), 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        #else:
        #    print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))
    
    
    
    #return target.data, label, correct
        

In [13]:
test_model(model, batch_size=20)

tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0])
tensor([[1., 1.],
        [2., 1.],
        [4., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 0.],
        [1., 1.],
        [1., 1.],
        [1., 0.],
        [4., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])
Test Loss: 0.041329

tensor([0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([[ 2.,  1.],
        [ 7.,  0.],
        [ 2.,  1.],
        [11.,  1.],
        [ 1.,  0.],
        [ 6.,  1.],
        [ 1.,  1.],
        [ 1.,  1.],
        [ 1.,  1.],
        [ 2.,  1.],
        [ 1.,  1.],
        [ 1.,  1.],
        [ 2.,  1.],
        [ 2.,  1.],
        [ 2.,  1.],
        [ 1.,  0.],
        [32.,  1.],
        [ 1.,  1.],
        [ 2.,  1.],
        [ 3.,  1.]])
tensor([0,

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [14]:
TaggedWord = namedtuple('TaggedWord', 'word freq dict tag')
tagged_words = [TaggedWord(w, f, d, c) for w,f,d,c 
                in test[["word", "frequency", "dict", "class"]].values]
for test in tagged_words:
    if test.word in w2v_model: 
        print( "단어 : " ,test.word, "\t실제 레이블 : " ,test.tag ,"\t예측 레이블 : ")

NameError: name 'w2v_model' is not defined